In [1]:
import pandas as pd
import altair as alt
import os

In [2]:
from ecostyles import EcoStyles
# Create styles instance
styles = EcoStyles()
# Register and enable a theme
styles.register_and_enable_theme(theme_name="article")  # or "article"

In [3]:
os.chdir('/Users/sambickel-barlow/RADataHub/RADataHub/Article Charts/UK_migration')

In [4]:
non_eu_migra = pd.read_excel('ECO - Chart data.xlsx', sheet_name='F1')

In [15]:
non_eu_migra_dep = non_eu_migra[non_eu_migra['Dependants'] == 'Including dependants']
non_eu_migra_dep = non_eu_migra_dep[~non_eu_migra_dep['Reason'].isin(['Total', 'Family'])]

In [41]:
non_eu_migra_dep['test'] = non_eu_migra_dep['Immigration'] / 1000

In [45]:
non_eu_migra_dep['Immigration_label'] = [f'{int(i / 1000)}K' for i in non_eu_migra_dep['Immigration']]

In [132]:
lines = alt.Chart(non_eu_migra_dep).mark_line().encode(
    x=alt.X('Year ending:T', axis=alt.Axis(labelFontSize=14, tickCount=5)),
    y=alt.Y('Immigration', axis=alt.Axis(labelExpr="datum.value / 1000 + 'K'", labelFontSize=14)),
    color='Reason',
    tooltip=[
        alt.Tooltip('Period', title='Quarter'),
        alt.Tooltip('Immigration_label', title='Immigrants'),
        alt.Tooltip('Reason', title='Reason for migration')
        
    ]
).properties(
    height=400,
    width=600
)

last_points = non_eu_migra_dep.loc[non_eu_migra_dep.groupby('Reason')['Year ending'].idxmax()]

# Manual adjustment parameters - ADJUST THESE AS NEEDED
label_adjustments = {
    'Study': 7500,      # No adjustment
    'Work': -7500,      # No adjustment  
    'Asylum': 0,      # No adjustment
    'Humanitarian': 0,      # No adjustment
    'Other': 0        # No adjustment
}

# Apply manual adjustments
last_points['adjusted'] = last_points.apply(
    lambda row: row['Immigration'] + label_adjustments.get(row['Reason'], 0), 
    axis=1
)
text = alt.Chart(last_points).mark_text(
    align='left',
    dx=5,
    fontWeight='bold',
    fontSize=14
).encode(
    x=alt.X('Year ending:T', axis=alt.Axis(tickCount=5)),
    y=alt.Y('adjusted'),
    text='Reason',
    color=alt.Color('Reason', legend=None)
)

chart1 = lines + text
chart1

alt.LayerChart(...)

In [176]:
# Save to png
chart1.save('UK_immigration_fig1.png', scale_factor=2)
# Save to json
chart1.save('UK_immigration_fig1.json', scale_factor=2)

In [133]:
jobs = pd.read_excel('ECO - Chart data.xlsx', sheet_name='F2')

In [135]:
jobs = jobs[jobs['Measure'] == 'Employee jobs held']

In [136]:
jobs['Date'] = pd.to_datetime(jobs['Period'], format='%B %Y')

/var/folders/qz/pj0lh7817m3c9ydwgfrtbtf00000gn/T/ipykernel_30614/1314184369.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobs['Date'] = pd.to_datetime(jobs['Period'], format='%B %Y')


In [138]:
jobs_melt = jobs.melt(id_vars=['Date'], value_vars=['Non-UK' , 'EU', 'Non-EU'], var_name='Employee Origin', value_name='Jobs Held')

In [140]:
jobs_melt['Jobs Held label'] = [f'{round(i / 1e6, 1)}M' for i in jobs_melt['Jobs Held']]

In [240]:
lines = alt.Chart(jobs_melt).mark_line().encode(
    x=alt.X('Date:T', axis=alt.Axis(labelFontSize=14, tickCount=5)),
    y=alt.Y('Jobs Held', axis=alt.Axis(labelExpr="datum.value / 1e6 + 'M'", labelFontSize=14)),
    color='Employee Origin',
    tooltip=[
        alt.Tooltip('Date', title='Date'),
        alt.Tooltip('Jobs Held label', title='Jobs Held'),
        alt.Tooltip('Employee Origin', title='Employee Origin')
    ]
).properties(
    height=400,
    width=600)
last_points = jobs_melt.loc[jobs_melt.groupby('Employee Origin')['Date'].idxmax()]

text = alt.Chart(last_points).mark_text(fontSize=14).encode(
    x=alt.X('Date:T'),
    y=alt.Y('Jobs Held', axis=alt.Axis(labelExpr="datum.value / 1e6 + 'M'")),
    text=alt.Text('Employee Origin'),
    color=alt.Color('Employee Origin', legend=None)
)

brexit_line = alt.Chart(pd.DataFrame({'Date': [pd.to_datetime('2020-10-01')]})).mark_rule(strokeWidth=2, strokeDash=[4,4], color='gray').encode(x='Date:T')
brexit_text = alt.Chart(pd.DataFrame({'Date': [pd.to_datetime('2020-10-01')], 'label': ['Post-Brexit immigration system introduced'] })).mark_text(
    align='left',
    dx=5,
    dy=-190,
    fontSize=11,
    fontWeight='bold',
    fontStyle='italic',
    color='gray'
).encode(
    x='Date:T',
    text='label'
)

chart2 = brexit_line + lines + text + brexit_text
chart2


alt.LayerChart(...)

In [177]:
# Save to png
chart2.save('UK_immigration_fig2.png', scale_factor=2)
# Save to json
chart2.save('UK_immigration_fig2.json', scale_factor=2)

In [178]:
fiscal = pd.read_excel('ECO - Chart data.xlsx', sheet_name='F3')

In [180]:
fiscal.columns

Index(['Age', 'Representative UK resident', 'Average-wage migrant worker',
       'High-wage migrant worker', 'Low-wage migrant worker'],
      dtype='object')

In [181]:
fiscal_melt = fiscal.melt(id_vars=['Age'], value_vars=['Representative UK resident', 'Average-wage migrant worker', 'High-wage migrant worker', 'Low-wage migrant worker'], var_name='Worker', value_name='Fiscal Impact')

In [291]:
fiscal_melt['Fiscal Impact label'] = [
    f'£{int(i / 1e3)}K' if pd.notna(i) else None 
    for i in fiscal_melt['Fiscal Impact']
]

In [292]:
lines = alt.Chart(fiscal_melt).mark_line().encode(
    x=alt.X('Age:Q', axis=alt.Axis(labelFontSize=14, grid=False, tickCount=20, title='Age', titleFontSize=14)),
    y=alt.Y('Fiscal Impact', axis=alt.Axis(labelFontSize=14, labelExpr=" '£' + (datum.value / 1e6) + 'M' ", title=None, titleFontSize=14)),
    color='Worker',
    tooltip=[
        alt.Tooltip('Age', title='Age'),
        alt.Tooltip('Worker', title='Worker Type'),
        alt.Tooltip('Fiscal Impact label', title='Fiscal Impact'),
        
    ]
).properties(
    height=400,
    width=600
)
last_points = fiscal_melt.loc[fiscal_melt.groupby('Worker')['Age'].idxmax()]

text = alt.Chart(last_points).mark_text(fontSize=14).encode(
    x=alt.X('Age:Q'),
    y=alt.Y('Fiscal Impact:Q'),
    text=alt.Text('Worker'),
    color=alt.Color('Worker', legend=None)
)

zero_line = alt.Chart(pd.DataFrame({'Fiscal Impact': [0]})).mark_rule(strokeWidth=2, color='lightgray').encode(y='Fiscal Impact:Q')

note1 = alt.Chart(pd.DataFrame({'Age': [45], 'label': ['Note: OBR assumes migrants move to the UK at age 25'] })).mark_text(
    align='right',
    dx=0,
    dy=433,
    fontSize=11,
    fontWeight='bold',
    fontStyle='italic',
    color='gray'
).encode(
    x='Age:Q',
    y=alt.value(0),
    text='label'
)

vert_line = alt.Chart(pd.DataFrame({'Age': [82]})).mark_rule(strokeWidth=2, strokeDash=[4,4], color='gray').encode(x='Age:Q')

note2 = alt.Chart(pd.DataFrame({'Age': [82], 'label': ['In 2021 UK life expectancy was 82 years old'] })).mark_text(
    align='left',
    dx=5,
    dy=5,
    fontSize=13,
    fontWeight='bold',
    fontStyle='italic',
    color='gray'
).encode(
    x='Age:Q',
    y=alt.value(0),
    text='label'
)

chart3 = zero_line + vert_line + lines + text + note1 + note2
chart3


alt.LayerChart(...)

In [293]:
# Save to png
chart3.save('UK_immigration_fig3.png', scale_factor=2)
# Save to json
chart3.save('UK_immigration_fig3.json', scale_factor=2)